<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/F1%20score%20metric%20with%20nasnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 13.4 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
     |████████████████████████████████| 168 kB 71.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 81.2 MB/s 
     |████████████████████████████████| 166 kB 75.0 MB/s 
     |████████████████████████████████| 166 kB 77.6 MB/s 
     |████████████████████████████████| 162 kB 62.4 MB/s 
     |████████████████████████████████| 162 kB 73.5 MB/s 
     |████████████████████████████████| 158 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 78.7 MB/s 
     |████████████████████████████████| 157 kB 77.3 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 157 kB 83.0 MB/s 
     |████████████████████████████████| 157 kB 73.0 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
nasnetMobile = tf.keras.applications.NASNetMobile(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_nasnetMobile=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  nasnetMobile,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
nasnetMobile.trainable=False
model_nasnetMobile.summary()

19993432/19993432 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 1056)              4269716   
                                                                 
 dropout (Dropout)           (None, 1056)              0         
                                                                 
 dense (Dense)               (None, 5)                 5285      
                                                                 
Total params: 4,275,001
Trainable params: 5,285
Non-trainable params: 4,269,716
_________________________________________________________________


In [7]:
from sklearn.metrics import f1_score
def f1_metrics(y_true, y_pred):
    return f1_score(y_true, y_pred, average = 'weighted')


In [17]:
def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + tf.keras.backend.epsilon())
    r = tp / (tp + fn + tf.keras.backend.epsilon())

    f1 = 2 * p * r / (p + r + tf.keras.backend.epsilon())
    f1 = tf.where(tf.math.is_finite(f1), f1, tf.zeros_like(f1))
    return tf.reduce_mean(f1)

In [18]:
model_nasnetMobile.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy', f1_score]
              )

In [9]:
wandb.init(
      # Set the project where this run will be logged
      project="Results", 
      name = "NASNet",
      )

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [19]:
%%time
epochs=1
history = model_nasnetMobile.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
# callbacks=[WandbCallback()]
)

22/22 [==============================] - 295s 9s/step - loss: 1.1577 - accuracy: 0.5810 - f1_score: 0.2086 - val_loss: 0.8221 - val_accuracy: 0.6971 - val_f1_score: 0.2720
CPU times: user 21.8 s, sys: 1.9 s, total: 23.7 s
Wall time: 5min 34s


In [20]:
wandb.finish()

In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4


In [22]:
model_nasnetMobile.evaluate(test_ds)

4/4 [==============================] - 27s 150ms/step - loss: 0.7881 - accuracy: 0.6900 - f1_score: 0.2776


[0.788059413433075, 0.6899999976158142, 0.2775593400001526]